In [1]:
from imutils import face_utils
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import imutils
import argparse 
import imutils
import time
import dlib
import cv2
import os 

In [2]:
# load the face detector (HOG-SVM)
print("[INFO] loading dlib thermal face detector...")
detector = dlib.simple_object_detector("../../../Documents/thermal-project-data/dlib_models/dlib_face_detector.svm")
#detector = dlib.simple_object_detector("G:/Other computers/My MacBook Air/thermal-project-data/dlib_models/dlib_face_detector.svm")

# load the facial landmarks predictor
print("[INFO] loading facial landmark predictor...")
predictor = dlib.shape_predictor("../../../Documents/thermal-project-data/dlib_models/dlib_landmark_predictor.dat")
#predictor = dlib.shape_predictor("G:/Other computers/My MacBook Air/thermal-project-data/dlib_models/dlib_landmark_predictor.dat")

[INFO] loading dlib thermal face detector...
[INFO] loading facial landmark predictor...


In [3]:
def get_length(filename):
    '''
    Input: Filename including path
    Returns: The total duration in seconds
    '''
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                             "format=duration", "-of",
                             "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return float(result.stdout)

In [4]:
# assume that we know the fps, this assumption is true for Flir Thermal camera (iOS model)
fps = 8.737238044062332

# This cell will find out the frame number corresponding to the timestamp of interest

# location fo the file
filenameWithPath = '../../../Documents/thermal-project-data/cycling/20221011T152141.MP4'

#filenameWithPath = "G:/Other computers/My MacBook Air/thermal-project-data/cycling/20221011T152141.MP4"

#"G:\Other computers\My MacBook Air\thermal-project-data\cycling"

totalFrames = fps*get_length(filenameWithPath)

# helper to find frame number from the timestamp
print("Total Frames In The Video Feed Is = {}".format(totalFrames))

# timestamp of interest should be given below.
# the corresponding frame number will be returned
minutes = 0
seconds = 5
totalSeconds = (minutes*60)+(seconds)
frameElapsed = totalSeconds*fps
print("Frame of interst is {} rounded to {}".format(frameElapsed, round(frameElapsed)))

Total Frames In The Video Feed Is = 1579.372270059108
Frame of interst is 43.68619022031166 rounded to 44


In [5]:
cap = cv2.VideoCapture(filenameWithPath)
# get the frame rate
fps = cap.get(cv2.CAP_PROP_FPS)

totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

# get the total frame as per opencv2
print("Total number of frames = {}".format(totalFrames))

# number of frame to fast forward. These many frames will be skipped.
# these number is close to the timestamp of interest where the event has occured.
fastForwardFrame = round(frameElapsed)-5;

fastForwardFrame = 0

# keep track of dropped frames
drop = 0

#print(fastForwardFrame)

# keep a tab of the frame count
frameNumber = 0 if fastForwardFrame == 0 else fastForwardFrame


# move frame that we want to fast forward
while fastForwardFrame:
    correct,frame = cap.read()
    fastForwardFrame-=1;


NosFrame = totalFrames
# this variable will be reduced after each frame is processed
frameLeft = NosFrame
#fig, ax = plt.subplots(round(np.sqrt(NosFrame)),round(np.sqrt(NosFrame)), figsize=(45,45))

j=0;i=0
correct = True
while frameLeft and correct:
    
    # read the first frame of the video
    correct,frame = cap.read()
    
    # change the channels so that we are matplotlib ready
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    
    try:
        # detect faces in the image 
        rects = detector(frame, upsample_num_times=1)

        (x, y, w, h) = face_utils.rect_to_bb(rects[0])
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        shape = predictor(frame, rects[0])
        shape = face_utils.shape_to_np(shape)
        
        for (sx, sy) in shape:
            cv2.circle(frame, (sx, sy), 2, (0, 0, 255), -1)
            

        cv2.imshow("frame", frame)
        key = cv2.waitKey(int((1/fps)*1000)) 
        if key == 27 or key == ord('q'):
            break
        #ax[j,i].imshow(frame)
    except Exception as e:
        drop+=1
        cv2.imshow("frame", frame)
        key = cv2.waitKey(int((1/fps)*1000)) 
        if key == 27 or key == ord('q'):
            break
        #print(e)
        continue
        #ax[j,i].imshow(frame)
    #i+=1;
    #if i==round(np.sqrt(NosFrame)):
    #    i=0
    #    j+=1
    frameLeft-=1; frameNumber+=1
cap.release()
cv2.destroyAllWindows()
print("Total Frames Dropped = {}".format((drop/totalFrames)*100))

Total number of frames = 1587.0


error: OpenCV(4.6.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [6]:
drop

784

In [7]:
784/1587

0.49401386263390046